In [1]:
# !pip install tensorflow-gpu==2.6.*
import tensorflow as tf
print(tf.__version__)

In [ ]:
!nvidia-smi
!nvcc --version

In [3]:
!pip install textstat xgboost gensim==4.2.0

In [4]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import re
import datetime
import time
import string
import statistics
import textstat
from bs4 import BeautifulSoup
from collections import Counter
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [5]:
import nltk
# nltk.download('punkt') If needed
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from bs4 import BeautifulSoup ## handles html
import re ## provides regular expressions functionality
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Concatenate, Flatten ,Dense, Embedding, LSTM, GRU, Bidirectional, BatchNormalization, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#read data
from google.colab import drive
drive.mount('/content/drive')

In [7]:
train = pd.read_csv("../input/adamssose22/train.csv", sep=",", encoding="utf-8")

In [8]:
test = pd.read_csv("../input/adamssose22/test.csv", sep=",", encoding="utf-8")

In [ ]:
train_review = pd.read_csv("drive/MyDrive/reviews.csv", sep=",", encoding="utf-8")

In [ ]:
train_review.info()

In [ ]:
train_review.head()

In [ ]:
train.columns


In [ ]:
train[train.host_id == train.host_id[2]]

In [ ]:
train.head()

In [ ]:
# Show target distribution
plt.hist(train.price)
plt.xlabel('Price')
plt.title('Price distribution')
plt.show();

In [ ]:
train.isnull().sum()/55284 * 100

In [ ]:

train['reviews_per_month'].describe()

In [ ]:
train_duplicate = train.copy()

In [ ]:
train_duplicate[train_duplicate.duplicated()]

In [ ]:
train.host_response_rate.value_counts()

In [ ]:
train['host_response_time'].isnull().sum()

In [ ]:

train['zipcode'].mode()

## Cleaning test dataset

In [ ]:
test['host_is_superhost'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
test.host_is_superhost[test.host_is_superhost == "t"] = 1 #replacing "t" with 1
test.host_is_superhost[test.host_is_superhost == "f"] = 0 #replacing "f" with 0
test["host_is_superhost"] = test["host_is_superhost"].astype("int64") #converting host_is_superhost to boolean datatype
test['host_has_profile_pic'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
test.host_has_profile_pic[test.host_has_profile_pic == "t"] = 1 #replacing "t" with 1
test.host_has_profile_pic[test.host_has_profile_pic == "f"] = 0 #replacing "f" with 0
test["host_has_profile_pic"] = test["host_has_profile_pic"].astype("int64") #converting host_is_superhost to boolean datatype

test['host_identity_verified'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
test.host_identity_verified[test.host_identity_verified == "t"] = 1 #replacing "t" with 1
test.host_identity_verified[test.host_identity_verified == "f"] = 0 #replacing "f" with 0
test["host_identity_verified"] = test["host_identity_verified"].astype("int64") #converting host_is_superhost to boolean datatyp
test['host_total_listings_count'].fillna(test['host_total_listings_count'].mean(),inplace = True) #replacing NA with mean of 'host_total_listings_count']
test['zipcode'].fillna('E1',inplace = True) #replacing NA with most frequently occuring zipcode 
test['bathrooms'].fillna(test['bathrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 
test['bedrooms'].fillna(test['bedrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 
test['beds'].fillna(test['beds'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

test['review_scores_rating'].fillna(test['review_scores_rating'].mean(),inplace = True)
test['review_scores_accuracy'].fillna(test['review_scores_accuracy'].mean(),inplace = True)
test['review_scores_cleanliness'].fillna(test['review_scores_cleanliness'].mean(),inplace = True)
test['review_scores_checkin'].fillna(test['review_scores_checkin'].mean(),inplace = True)
test['review_scores_communication'].fillna(test['review_scores_communication'].mean(),inplace = True)
test['review_scores_location'].fillna(test['review_scores_location'].mean(),inplace = True)
test['review_scores_value'].fillna(test['review_scores_value'].mean(),inplace = True)
test['reviews_per_month'].fillna(test['reviews_per_month'].mean(),inplace = True)

test['summary'].fillna('empty',inplace = True)
test["property_type"] = test["property_type"].astype("category") 
test["room_type"] = test["room_type"].astype("category") 
test["bed_type"] = test["bed_type"].astype("category") 
test['host_response_time'].fillna("missing",inplace = True)
test["host_response_time"] = test["host_response_time"].astype("category")



In [ ]:
test['summary'].fillna('empty',inplace = True)
test['name'].fillna('empty',inplace = True)
test['space'].fillna('empty',inplace = True)
test['description'].fillna('empty',inplace = True)
test['neighborhood_overview'].fillna('empty',inplace = True)
test['transit'].fillna('empty',inplace = True)
test['house_rules'].fillna('empty',inplace = True)

In [ ]:
test["host_since"] = test["host_since"].astype("datetime64[ns]")
test["host_since"].fillna(pd.to_datetime('2015-05-21'),inplace = True)
test['host_days'] = (datetime.datetime.today() - test['host_since'])//np.timedelta64(1,'D')   
test['host_days'] = test['host_days']/365
x_test_dummies = pd.get_dummies(test, columns = ['host_response_time','room_type','bed_type'], drop_first=True)          
x_test_dummies = x_test_dummies.select_dtypes(exclude=["category","datetime64[ns]","object"])
x_test_dummies.drop(["host_id","latitude","longitude"], inplace=True, axis=1)

## Cleaning Train Dataset

In [ ]:
train['host_is_superhost'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
train.host_is_superhost[train.host_is_superhost == "t"] = 1 #replacing "t" with 1
train.host_is_superhost[train.host_is_superhost == "f"] = 0 #replacing "f" with 0
train["host_is_superhost"] = train["host_is_superhost"].astype("int64") #converting host_is_superhost to boolean datatype
train['host_has_profile_pic'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
train.host_has_profile_pic[train.host_has_profile_pic == "t"] = 1 #replacing "t" with 1
train.host_has_profile_pic[train.host_has_profile_pic == "f"] = 0 #replacing "f" with 0
train["host_has_profile_pic"] = train["host_has_profile_pic"].astype("int64") #converting host_is_superhost to boolean datatype

train['host_identity_verified'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
train.host_identity_verified[train.host_identity_verified == "t"] = 1 #replacing "t" with 1
train.host_identity_verified[train.host_identity_verified == "f"] = 0 #replacing "f" with 0
train["host_identity_verified"] = train["host_identity_verified"].astype("int64") #converting host_is_superhost to boolean datatyp


In [ ]:
train['bathrooms'].isnull().sum()

In [ ]:
train['host_total_listings_count'].fillna(train['host_total_listings_count'].mean(),inplace = True) #replacing NA with mean of 'host_total_listings_count']

In [ ]:
train['neighbourhood'].mode()


In [ ]:
train['neighbourhood'].fillna('City of Westminster',inplace = True) #replacing NA with most frequently occuring neighbourhood

In [ ]:
train['zipcode'].mode()

In [ ]:
train['zipcode'].fillna('E1',inplace = True) #replacing NA with most frequently occuring zipcode 

In [ ]:
train['bathrooms'].fillna(train['bathrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

In [ ]:
train['bedrooms'].fillna(train['bedrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

In [ ]:
train['beds'].fillna(train['beds'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

In [ ]:
train['review_scores_rating'].fillna(train['review_scores_rating'].mean(),inplace = True)
train['review_scores_accuracy'].fillna(train['review_scores_accuracy'].mean(),inplace = True)
train['review_scores_cleanliness'].fillna(train['review_scores_cleanliness'].mean(),inplace = True)
train['review_scores_checkin'].fillna(train['review_scores_checkin'].mean(),inplace = True)
train['review_scores_communication'].fillna(train['review_scores_communication'].mean(),inplace = True)
train['review_scores_location'].fillna(train['review_scores_location'].mean(),inplace = True)
train['review_scores_value'].fillna(train['review_scores_value'].mean(),inplace = True)
train['reviews_per_month'].fillna(train['reviews_per_month'].mean(),inplace = True)


In [ ]:
train['summary'].fillna('empty',inplace = True)
train["property_type"] = train["property_type"].astype("category") 
train["room_type"] = train["room_type"].astype("category") 
train["bed_type"] = train["bed_type"].astype("category") 
train['host_response_time'].fillna("missing",inplace = True)
train["host_response_time"] = train["host_response_time"].astype("category") 


In [ ]:
train['summary'].fillna('empty',inplace = True)
train['name'].fillna('empty',inplace = True)
train['space'].fillna('empty',inplace = True)
train['description'].fillna('empty',inplace = True)
train['neighborhood_overview'].fillna('empty',inplace = True)
train['transit'].fillna('empty',inplace = True)
train['house_rules'].fillna('empty',inplace = True)

In [ ]:
train["host_since"].mode()

In [ ]:
import pickle

train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

## Cleaning Text Data

In [ ]:
def remove_whitespace(text):
    """ Function to remove whitespace (tabs, newlines). """
    return ' '.join(text.split())

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def remove_punctuation_and_casing(text):
    """
    Function to remove the punctuation, upper casing and words that include
    non-alphanumeric characters.
    """
    chars = '!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    text = text.translate(str.maketrans(chars, ' ' * len(chars)))
    return ' '.join([word.lower() for word in text.split() if word.isalpha()])

In [ ]:
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

english_stopwords = stopwords.words('english')

In [ ]:
def remove_stopwords(text):
    """ Function to remove stopwords. """
    return ' '.join([word for word in str(text).split() if word not in english_stopwords])

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Helper function that calls the POS tagger for an input word and return a code that can be used for lemmatization"""
    # Extract the first letter of the POS tag (see the above example to understand the output coming from pos_tag)
    tag = nltk.pos_tag([word])[0][1][0].upper()  
    # Dictionary to map these letters to wordnet codes that the lemmatizer understands
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def text_cleaning(documents):
    """
    Function for standard NLP pre-processing including removal of html tags,
    whitespaces, non-alphanumeric characters, and stopwords. Emoticons are
    converted to text that reflects their meaning. Words are subject to
    lemmatization using their POS tags.
    """
    cleaned_text = []  # our output will be a list of documents
    lemmatizer = WordNetLemmatizer()
    
    print('Processing input array with {} elements...'.format(documents.shape[0]))
    counter = 0
    
    for doc in documents:
        text = BeautifulSoup(doc).get_text() # remove html content
        text = remove_whitespace(text) # remove whitespaces
        text = remove_punctuation_and_casing(text) # remove punctuation and casing
        text = remove_stopwords(text) # remove stopwords
        text = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text.split()]) # lemmatize each word
        
        cleaned_text.append(text)

        if (counter > 0 and counter % 50 == 0):
            print('Processed {} documents'.format(counter))
            
        counter += 1
        
    return cleaned_text


In [ ]:
nltk.download('stopwords') ## to identify stopwords 
nltk.download('averaged_perceptron_tagger') ## for part-of-speech tagging (used for lemmatization)
nltk.download('omw-1.4')
nltk.download('wordnet')

# Lemmatize with POS Tag (Parts of Speech tagging)
def get_wordnet_pos(word):
    """Map POS tag to first character for lemmatization"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

## function to clean text data
def clean_reviews(df):
    """ Standard NLP pre-processing chain including removal of html tags, non-alphanumeric characters, and stopwords.
        Words are subject to lemmatization using their POS tags, which are determind using WordNet. 
    """
    reviews = []

    lemmatizer = WordNetLemmatizer()
    
    print('*' * 40)
    print('Cleaning {} movie reviews.'.format(df.shape[0]))
    counter = 0
    for review in df:
        
        # remove html content
        review_text = BeautifulSoup(review).get_text()
        
        # remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        # tokenize the sentences with all capital letters transformed to lower case
        words = word_tokenize(review_text.lower())
  
        # filter stopwords
        words = [w for w in words if w not in stopwords.words("english")]
        
        # lemmatize each word to its lemma
        lemma_words =[lemmatizer.lemmatize(i, get_wordnet_pos(i)) for i in words]
    
        reviews.append(lemma_words)
              
        if (counter > 0 and counter % 5000 == 0):
            print('Processed {} reviews'.format(counter))
            
        counter += 1
        
    print('DONE')
    print('*' * 40)

    return(reviews) 

In [ ]:
nltk.download('punkt')
summary_clean = clean_reviews(train.summary)

In [ ]:
train["summary_clean"] = summary_clean

In [ ]:
train.summary_clean

In [ ]:
train.summary

In [ ]:
#summary	space	description, neighborhood_overview	transit	house_rules
#'name','summary','space','description','experiences_offered','neighborhood_overview','transit','house_rules'
text_processed = text_cleaning(train.experiences_offered)
print(text_processed)

# Modeling

### using only numerical data and FNN

In [ ]:
train["host_since"].mode()

In [ ]:
train["host_since"] = train["host_since"].astype("datetime64[ns]")
train["host_since"].fillna(pd.to_datetime('2015-05-21'),inplace = True)
train['host_days'] = (datetime.datetime.today() - train['host_since'])//np.timedelta64(1,'D')   
train['host_days'] = train['host_days']/365

In [ ]:
x_train_dummies = pd.get_dummies(train, columns = ['host_response_time','room_type','bed_type'], drop_first=True)          
x_train_dummies = x_train_dummies.select_dtypes(exclude=["category","datetime64[ns]","object"])
x_train_dummies.drop(["host_id","latitude","longitude"], inplace=True, axis=1)

In [ ]:
x_train_dummies["summary_clean"] = summary_clean

In [ ]:
x_train_dummies.info()

In [ ]:
y = x_train_dummies[['price']]
x_train_dummies.pop("price")

In [ ]:
x_train_dummies.drop(["summary_clean"], inplace=True, axis=1)

In [ ]:
seed=888
X_train, X_test, y_train, y_test = train_test_split(x_train_dummies, y, test_size=0.30, random_state=seed)
    
print (X_train.shape)
print(y_train.shape) 

In [ ]:
nb_units = 128

In [ ]:
 y.shape

In [ ]:
x_train_dummies.shape[1]

In [ ]:
inputs = layers.Input(shape=(x_train_dummies.shape[1],))
layer1 = layers.Dense(nb_units,  activation='relu', #activation function
                kernel_initializer=keras.initializers.he_normal(seed=seed), #initialisation of weights, 
                bias_initializer='zeros')(inputs)
layer2 = layers.Dense(nb_units,activation='relu',kernel_initializer=keras.initializers.he_normal(seed=seed),bias_initializer='zeros')(layer1)
predictions = layers.Dense(1, activation='relu')(layer2)

model = keras.Model(inputs=inputs, outputs=predictions)
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae']) 
    
history = model.fit(
    X_train, 
    y_train, 
    epochs = 5, 
    batch_size=1,
    validation_data=(X_test, y_test),
    verbose = 0) 

## make predictions
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_test)

In [ ]:
?model.compile

In [ ]:
pred_ff_test = model.predict(X_test)

In [ ]:
pred_ff_test = model.predict(x_test_dummies)

In [ ]:
test.info()

In [ ]:
pred_ff_test.shape

In [ ]:
loss = history.history["mae"] 
val_loss = history.history["val_mae"] 
epochs = range(1, len(loss) + 1) 
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from IPython.display import FileLink, FileLinks

In [ ]:
pred_kaggle = pd.DataFrame(pred_ff_test, columns=['price'], index = test.listing_id)
pred_kaggle.to_csv('C:/Users/ASUS/Desktop/pred_kaggle.csv')

In [ ]:
mean_squared_error(y_train,pred_ff_train)
pred_ff_test = model.predict(y_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,pred_ff_test)))

## modeling using only summary column and fnn

In [ ]:
import io
import re
import string
import pandas as pd
import gensim
from gensim.models import Word2Vec  
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
Xclean_train, Xclean_val, y_train, y_val = train_test_split(train['summary'], train['price'], test_size = 0.2, random_state = 888)

In [ ]:
X_train = Xclean_train.to_numpy()
X_val = Xclean_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [ ]:
Xclean_train.shape

In [ ]:
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

###### Emergency back up of data to put in NN

In [ ]:


X_train_bk = X_train.copy()
y_train_bk = y_train.copy()
X_test_bk = X_test.copy()
y_test_bk = y_test.copy()

In [ ]:
# Create a simple model to use word embeddings
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def make_model():
  # Define the size of the vocabulary and the max number of words in a sequence
  vocab_size = 10000
  seq_length = 100

  # Create a vectorization layer
  vectorize_layer = TextVectorization(
      standardize = our_standardization,
      max_tokens = vocab_size,
      output_sequence_length = seq_length
      )

  vectorize_layer.adapt(X_train)
  embedding_dim = 16
  model = Sequential([
                    vectorize_layer,
                    layers.Embedding(vocab_size, embedding_dim, name="embedding"), 
                    layers.GlobalAveragePooling1D(), # each sample is reduced to the average of the word embeddings
                    layers.Dense(1, activation='relu')
                  ])
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae','mse'])
  return model


model = make_model()
  
  
  
history = model.fit(
      X_train, 
      y_train, 
      epochs = 5, 
      batch_size=64,
      validation_data=(X_test, y_test),
      verbose = 1
      ) 


In [ ]:
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_val)

In [ ]:
pred_ff_test

In [ ]:
pred_ff_test

In [ ]:
y_train

In [ ]:
y_val

In [ ]:
X_val

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val,pred_ff_test)))

In [ ]:
test_summary = test['summary'].to_numpy()

In [ ]:
test_summary.shape

In [ ]:
pred_ff_kaggle = model.predict(test_summary)

In [ ]:
pred_kaggle_summary = pd.DataFrame(pred_ff_kaggle, columns=['price'], index = test.listing_id)
pred_kaggle_summary.to_csv('C:/Users/ASUS/Desktop/pred_kaggle_summary.csv')

## LSTM

In [ ]:
vocab_size = 10000
seq_length = 500

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )
vectorize_layer.adapt(X_train)
## Transform sequences of words to seq of integers and labels to tensor

In [ ]:
X_train = vectorize_layer(X_train)
X_val = vectorize_layer(X_val)

In [ ]:
test_summary = vectorize_layer(test_summary)

In [ ]:
# Create model with LSTM
emb_size = 32
rnn_units = 16
input = tf.keras.Input(shape=(seq_length,), dtype="int64") 
emb = layers.Embedding(input_dim=vocab_size, output_dim=emb_size, mask_zero=True)(input) 
x = layers.LSTM(rnn_units)(emb)
output = layers.Dense(1, activation="relu")(x)
model = tf.keras.Model(input, output) 

model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae'])
model.summary()

In [ ]:
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)

In [ ]:
story = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    epochs = 30,
    batch_size = 128)

In [ ]:
X_train

In [ ]:
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_val)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val,pred_ff_test)))

In [ ]:
pred_ff_train

In [ ]:
pred_ff_test

In [ ]:
pred_ff_kaggle = model.predict(test_summary)

In [ ]:
pred_kaggle_summary = pd.DataFrame(pred_ff_kaggle, columns=['price'], index = test.listing_id)
pred_kaggle_summary.to_csv('C:/Users/ASUS/Desktop/pred_kaggle_summary.csv')

## LSTM

In [71]:
train_desc = train['description'].fillna('empty')
train_ngbr = train['neighborhood_overview'].fillna('empty')

y_train = train['price']

In [72]:
print(len(train_desc), len(train_ngbr), len(y_train))

In [11]:
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

In [12]:
our_standardization(train_desc[0])

In [13]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vocab_size = 10000
seq_length = 500

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )

In [14]:
desc1, desc2 = train_desc[2], train_desc[3]
print(desc1+'\n\n'+desc2)

ngbr1, ngbr2 = train_ngbr[2], train_ngbr[3]
print(ngbr1+'\n\n'+ngbr2)

In [17]:
!nvidia-smi

tf.test.is_gpu_available()

In [64]:
emb_size = 32
rnn_units = 16


def create_text_model(text_list):
    model = Sequential()
    model.add(Input(shape=(1,), dtype=tf.string))
    vectorize_layer.adapt(text_list)
    model.add(vectorize_layer)
    model.add(Embedding(input_dim=vocab_size, output_dim=emb_size))
    model.add(LSTM(rnn_units, return_sequences=False))
    model.add(Dense(1, activation="relu"))
    return model

def create_combined_model(X):
    X = Flatten()(X)
    X = Dense(1, activation="linear")(X)
    print(X.shape)
    return X

def create_model():
    
    ### Initialize Input layers
    input_desc = Input(shape=(1,), dtype=tf.string)
    input_ngbr = Input(shape=(1,), dtype=tf.string)
    
    ### Create Vectorisation models from text features
    desc_model = create_text_model(train_desc)
    ngbr_model = create_text_model(train_ngbr)
#     combined_model = create_combined_model()
    
    ### Create Data flow
    emb_desc = desc_model(input_desc)
    emb_ngbr = ngbr_model(input_ngbr)
    concat_combined = Concatenate()([emb_desc,emb_ngbr])
    print(concat_combined.shape)
    output = create_combined_model(concat_combined)
    
    ### Finalize the model
    model = Model(inputs = [input_desc, input_ngbr], outputs = output)
    model.compile(optimizer='adam',loss='mse', metrics=['mae'])
    return model








# desc_embedded = desc_model.predict([desc1,desc2])
# ngbr_model = create_text_model(train_ngbr)

# dense_desc = desc_model([desc2])
# dense_ngbr = ngbr_model([ngbr2])

# concatenated_values = layers.Concatenate([dense_model, ngbr_model])

# print(concatenated_values.shape)

# def create_model(text_features_list):
#   vec_model = 
#   for feature in text_features_list:
    


In [ ]:
# Create model with LSTM
emb_size = 100
rnn_units = 64

input_ngbr = tf.keras.Input(shape=(seq_length,), dtype="int64") 
input_desc = tf.keras.Input(shape=(seq_length,), dtype="int64") 
emb_desc = layers.Embedding(input_dim=vocab_size, output_dim=emb_size)(input_desc) 
x_desc = layers.GRU(rnn_units)(emb_desc)
dense_desc = layers.Dense(1, activation="relu")(x_desc)

emb_ngbr = layers.Embedding(input_dim=vocab_size, output_dim=emb_size)(input_ngbr) 
x_ngbr = layers.GRU(rnn_units)(emb_ngbr)
dense_ngbr = layers.Dense(1, activation="relu")(x_ngbr)

concat = concatenate([dense_desc, dense_ngbr])
norm = layers.BatchNormalization()(concat)
dense_full = Dense(128, activation="relu")(norm)
#dense_full = Dense(64, activation="relu")(dense_full)
output_layer = Dense(1, activation="relu")(dense_full)


model = tf.keras.Model(inputs = [input_desc, input_ngbr], outputs = output_layer) 

model.compile(optimizer='adam', 
    loss='mse', 
    metrics=['mae']) 
model.summary()

In [62]:
# tf.convert_to_tensor([zip(train_desc,train_ngbr))
y_train = tf.convert_to_tensor(y_train)

In [70]:
train_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(train_features.values, tf.string)
) 
train_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(train_targets.values, tf.int64),

) 


def convert_text_input(sample):
    text = sample
    text = tf.expand_dims(text, -1)  
    #return tf.squeeze(vectorize_layer(text))
    return tf.squeeze(vectorize_layer(text))

convert_text_input(["what is this misery","what is your story"]).shape

In [74]:
# model = create_model()
# desc_model = create_text_model(train_desc)

# print(desc_model.summary())
# desc_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
story = model.fit(
   [train_desc, train_ngbr],
   y_train,
#     validation_split=0.2,
    epochs = 5,
    batch_size = 32,
    verbose =1)

In [ ]:
?model.fit

In [75]:
preds = pd.DataFrame(model.predict([train_desc,train_ngbr]), columns=["price"], index=train.index)

In [76]:
preds

In [ ]:
?model.predict

In [ ]:
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_val)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val,pred_ff_test)))

In [ ]:
test_summary = vectorize_layer(test_summary)


In [ ]:
pred_ff_kaggle = model.predict(test_summary)

In [ ]:
pred_ff_kaggle

In [ ]:
pred_ff_train

In [ ]:
vocab_size = 10000
seq_length = 500
input = tf.keras.Input(shape=(seq_length,), dtype="int64") 
emb = layers.Embedding(input_dim=vocab_size, output_dim=emb_size, mask_zero=True)(input) 
x = layers.Bidirectional(layers.LSTM(rnn_units))(emb)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="relu")(x)
model = tf.keras.Model(input, output) 

model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae']) 
model.summary()

In [ ]:
model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    epochs = 2,
    batch_size = 128)